In [1]:
import torch
import numpy as np

In [2]:
# in this example:
# MAX_NUMBER_NODES = 4
# MIN_NUMBER_NODES = 2
# in the real dataset are 9 and 7, max and min respectively
a = torch.tensor([[1,0,1,1], [1,1,0,0], [1,0,-1,-1]]).type(torch.long)

In [3]:
def apply_mask_to_logits(logits, mask, idxs): 
  batch_size = logits.size(0)
  clone_mask = mask.clone()

  if idxs is not None:
      clone_mask[[i for i in range(batch_size)], idxs.data] = 1
      # logits[clone_mask] = -np.inf
      logits[clone_mask] = -1000000000
  return logits, clone_mask

In [4]:
batch_size = a.shape[0]
seq_len = a.shape[1]

In [5]:
# config 1:
a = torch.tensor([[1,0,1,1], [1,1,0,0], [1,0,-1,-1]]).type(torch.long)
mask = torch.zeros(batch_size, seq_len)
idxs = torch.tensor([2,3])

try:
  apply_mask_to_logits(a, mask, idxs)
except IndexError as e:
  print('Error shape mismatch')
  print(e)

Error shape mismatch
shape mismatch: indexing tensors could not be broadcast together with shapes [3], [2]


In [6]:
# config 2:
a = torch.tensor([[1,0,1,1], [1,1,0,0], [1,0,-1,-1]]).type(torch.long)
mask = torch.zeros(batch_size, seq_len).byte()
idxs = torch.tensor([1,2,3])

result, clone_mask = apply_mask_to_logits(a, mask, idxs)
print(result.shape)
print(result)
print("Meaningless result")

torch.Size([3, 4])
tensor([[          1, -1000000000,           1,           1],
        [          1,           1, -1000000000,           0],
        [          1,           0,          -1, -1000000000]])
Meaningless result


C:\Users\Felipe\AppData\Local\Temp\ipykernel_5720\3597876467.py:8: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead. (Triggered internally at  ..\aten\src\ATen\native\TensorAdvancedIndexing.cpp:1273.)
  logits[clone_mask] = -1000000000


In [7]:
# config 3:
a = torch.tensor([[1,0,1,1], [1,1,0,0], [1,0, -1, -1]]).type(torch.long)
mask = torch.zeros(batch_size, seq_len).byte()
idxs = torch.tensor([2])

result, clone_mask = apply_mask_to_logits(a, mask, idxs)
print(result.shape)
print(result)
print("Meaningless result")

torch.Size([3, 4])
tensor([[          1,           0, -1000000000,           1],
        [          1,           1, -1000000000,           0],
        [          1,           0, -1000000000,          -1]])
Meaningless result


C:\Users\Felipe\AppData\Local\Temp\ipykernel_5720\3597876467.py:8: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead. (Triggered internally at  ..\aten\src\ATen\native\TensorAdvancedIndexing.cpp:1273.)
  logits[clone_mask] = -1000000000


In [8]:
a = torch.tensor([[1,0,1,1,1], [1,1,0,0,1], [1,0,1,2,2], [0,1,0,1,1], [0,2,2,2,2], [0,1,1,2,2]]).type(torch.long)
batch_size = a.shape[0]
seq_len = a.shape[1]

mask = torch.zeros(batch_size, seq_len).byte()
idxs = [(2, 3), (4, 1), (5, 3)]

# in the real dataset this value is currently 36 (9 nodes max) (2022-07-26)
DATASET_SEQ_LEN = seq_len

def apply_mask_to_logits_2(logits, mask, idxs): 
  clone_mask = mask.clone()

  print(clone_mask.shape)

  if idxs is not None:
    for (idx, seq_len) in idxs:
      clone_mask[idx, [range(seq_len, DATASET_SEQ_LEN)]] = 1
    logits[clone_mask] = -1000000000
  return logits, clone_mask
    
result, clone_mask = apply_mask_to_logits_2(a, mask, idxs)
result

"""
Debugging the PointerNetwork.py, when the network gets a batch of length 2 with the first sample of 7 nodes
and the second sample of 9 nodes, the apply_mask_to_logits function returns the following:

tensor([[-0.1323, -0.1097,  0.0407,  0.0161,  0.1157,  0.1710,  0.1186,  0.0824,
          0.1728,  0.1248,  0.1953,  0.1445,  0.1093,  0.1947,  0.2370,  0.1772,
          0.1361,  0.1116,  0.0968,  0.0875,  0.0812,    -inf,    -inf,    -inf,
            -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,
            -inf,    -inf,    -inf,    -inf],
        [-0.0518, -0.0309, -0.0167, -0.0057,  0.1380,  0.1084,  0.2001,  0.1577,
          0.1287,  0.1132,  0.1047,  0.1000,  0.2137,  0.2690,  0.2130,  0.1775,
          0.1569,  0.1451,  0.1378,  0.2435,  0.2024,  0.1740,  0.1589,  0.1500,
          0.1444,  0.2489,  0.2090,  0.1805,  0.1651,  0.2599,  0.2194,  0.1892,
          0.2717,  0.2295,  0.2902,  0.2432]], grad_fn=<AsStridedBackward0>)
torch.Size([2, 36])
"""

torch.Size([6, 5])


C:\Users\Felipe\AppData\Local\Temp\ipykernel_5720\447886108.py:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead. (Triggered internally at  ..\aten\src\ATen\native\TensorAdvancedIndexing.cpp:1273.)
  logits[clone_mask] = -1000000000


tensor([[          1,           0,           1,           1,           1],
        [          1,           1,           0,           0,           1],
        [          1,           0,           1, -1000000000, -1000000000],
        [          0,           1,           0,           1,           1],
        [          0, -1000000000, -1000000000, -1000000000, -1000000000],
        [          0,           1,           1, -1000000000, -1000000000]])

In [11]:
# config 4:
# in this example:
# possible lenghts = [4, 2, 1]
# MAX_NUMBER_NODES = 5
# MIN_NUMBER_NODES = 1
# mask is not -1, it is actually 2

a = torch.tensor([[1,0,1,1,1], [1,1,0,0,1], [1,0,1,2,2], [0,1,0,1,1], [0,2,2,2,2], [0,1,1,2,2]]).type(torch.long)
batch_size = a.shape[0]
dataset_seq_len = a.shape[1]

masks = (a == 2).nonzero()
idxs = {}
for (idx, seq_len) in masks:
  idx = int(idx.cpu())
  if idx not in idxs:
    idxs[idx] = int(seq_len.cpu())
idxs = idxs.items()

if not len(idxs):
  idxs = None

mask = torch.zeros(batch_size, dataset_seq_len).bool()
# use .bool instead of .byte because of a depreceated warning

print(mask.shape)

result, clone_mask = apply_mask_to_logits_2(a, mask, idxs)
print(result.shape)
print(result)
print("Result with actual meaning")

torch.Size([6, 5])
torch.Size([6, 5])
torch.Size([6, 5])
tensor([[          1,           0,           1,           1,           1],
        [          1,           1,           0,           0,           1],
        [          1,           0,           1, -1000000000, -1000000000],
        [          0,           1,           0,           1,           1],
        [          0, -1000000000, -1000000000, -1000000000, -1000000000],
        [          0,           1,           1, -1000000000, -1000000000]])
Meaningless result


C:\Users\Felipe\AppData\Local\Temp\ipykernel_5720\447886108.py:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead. (Triggered internally at  ..\aten\src\ATen\native\TensorAdvancedIndexing.cpp:1273.)
  logits[clone_mask] = -1000000000


In [ ]:
# The problem with nn.Embedding

a = torch.tensor([[1,1,1,0], [1,1,0,1]])
embedding2 = torch.nn.Embedding(2, 32)
embedding4 = torch.nn.Embedding(4, 32)
print(embedding2(a).shape)
print(embedding4(a).shape)

a = torch.tensor([[1,1,1,0,0], [1,1,0,1,-1]])
embedding2 = torch.nn.Embedding(3, 32)
embedding4 = torch.nn.Embedding(5, 32)

try:
  print(embedding2(a).shape)
except IndexError as e:
  print('IndexError')
  print(e)

try:
  print(embedding4(a).shape)
except IndexError as e:
  print('IndexError2')
  print(e)

a = torch.tensor([[1,1,1,0,2], [1,1,0,1,3]])
embedding2 = torch.nn.Embedding(4, 32)
embedding4 = torch.nn.Embedding(5, 32)
print(embedding2(a).shape)
print(embedding4(a).shape)

torch.Size([2, 4, 32])
torch.Size([2, 4, 32])
IndexError
index out of range in self
IndexError2
index out of range in self
torch.Size([2, 5, 32])
torch.Size([2, 5, 32])
